<a href="https://colab.research.google.com/github/mahapara2411/Sentiment-Analysis/blob/master/Deep_Learning_Recommendation_System_With_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
#Upload the Dataset
from google.colab import files
uploaded = files.upload()

Saving Hotel Data1.csv to Hotel Data1.csv


In [3]:
# Dataset is now stored in a Pandas Dataframe
import io
data = pd.read_csv(io.BytesIO(uploaded['Hotel Data1.csv']))

In [4]:
data.head()

,Hotel_id,Hotel,Address,City,Title,Users,Reviews,Review_date,Rating,Hotel_Type,Price,Hotel_Category
0,AVwc252WIN2L1WUfpqLP,Rancho Valencia Resort Spa,5921 Valencia Cir,Rancho Santa Fe,Best romantic vacation ever!!!!,Paula,Our experience at Rancho Valencia was absolute...,2013-11-14T00:00:00Z,5.0,5,Rs.60332,"Hotels,Hotels and motels,Hotel and motel reser..."
1,AVwc252WIN2L1WUfpqLP,Rancho Valencia Resort Spa,5921 Valencia Cir,Rancho Santa Fe,Sweet sweet serenity,D,Amazing place. Everyone was extremely warm and...,2014-07-06T00:00:00Z,5.0,5,Rs.60332,"Hotels,Hotels and motels,Hotel and motel reser..."
2,AVwc252WIN2L1WUfpqLP,Rancho Valencia Resort Spa,5921 Valencia Cir,Rancho Santa Fe,Amazing Property and Experience,Ron,We booked a 3 night stay at Rancho Valencia to...,2015-01-02T00:00:00Z,5.0,5,Rs.60332,"Hotels,Hotels and motels,Hotel and motel reser..."
3,AVwdOclqIN2L1WUfti38,Aloft Arundel Mills,7520 Teague Rd,Hanover,"Never again...beware, if you want sleep.",jaeem2016,Currently in bed writing this for the past hr ...,2016-05-15T00:00:00Z,2.0,3,Rs.7873,"Hotels,Hotels and motels,Travel agencies and b..."
4,AVwdOclqIN2L1WUfti38,Aloft Arundel Mills,7520 Teague Rd,Hanover,ALWAYS GREAT STAY...,MamaNiaOne,I live in Md and the Aloft is my Home away fro...,2016-07-09T00:00:00Z,5.0,3,Rs.7873,"Hotels,Hotels and motels,Travel agencies and b..."


In [5]:
#Filing the missing Review with 'na'
data['Reviews'] = data['Reviews'].fillna('').apply(str)
data['Users'] = data['Users'].fillna('').apply(str)
data['Review_date'] = data['Review_date'].fillna('').apply(str)
data['Hotel_Type'] = data['Hotel_Type'].fillna('').apply(str)
data['Price'] = data['Price'].fillna('').apply(str)
data['Title'] = data['Title'].fillna('').apply(str)
data['Hotel_id'] = data['Hotel_id'].fillna('').apply(str)


In [6]:
data = data.dropna()
data = data.reset_index(drop=True)
print('Shape of dataset ',data.shape)
print(data.columns)


Shape of dataset  (24261, 12)
Index(['Hotel_id', 'Hotel', 'Address', 'City', 'Title', 'Users', 'Reviews',
       'Review_date', 'Rating', 'Hotel_Type', 'Price', 'Hotel_Category'],
      dtype='object')


In [7]:
g = data.groupby('Users')['Rating'].count()
top_users = g.sort_values(ascending=False)[:15]

In [9]:
top_users

Users
A verified traveler    1340
A Traveler              952
Anonymous               164
John                     59
Robert                   51
David                    46
Michael                  41
Mary                     28
Lisa                     25
James                    23
Mark                     22
Richard                  21
Jennifer                 20
William                  16
Christine                16
Name: Rating, dtype: int64

In [11]:
g = data.groupby('Hotel')['Rating'].count()
top_hotels = g.sort_values(ascending=False)[:15]

In [12]:
top_hotels

Hotel
Best Western Plus Waterville Grand Hotel              317
Doubletree By Hilton West Palm Beach Airport          276
Hyatt House Seattle/Downtown                          259
Doubletree By Hilton Hotel Bay City - Riverfront      227
Metro Points Hotel-Washington North                   202
Hotel Emma                                            183
The Westin Las Vegas Hotel & Spa                      171
Best Western Springfield                              161
Best Western Seven Seas                               151
French Market Inn                                     144
ARIA Resort Casino                                    139
Kinzie Hotel                                          138
St. James Hotel, an Ascend Hotel Collection Member    136
Grand Hyatt Seattle                                   135
Luana Waikiki Hotel & Suites                          132
Name: Rating, dtype: int64

In [15]:
top_r = data.join(top_users, rsuffix='_r', how='inner', on='Users')
top_r = top_r.join(top_hotels, rsuffix='_r', how='inner', on='Hotel')

In [17]:
pd.crosstab(top_r.Users, top_r.Hotel, top_r.Rating, aggfunc=np.sum)

Hotel,Best Western Plus Waterville Grand Hotel,Best Western Seven Seas,Best Western Springfield,Doubletree By Hilton Hotel Bay City - Riverfront,Doubletree By Hilton West Palm Beach Airport,Kinzie Hotel,Metro Points Hotel-Washington North,The Westin Las Vegas Hotel & Spa
Users,,,,,,,,
A Traveler,49.0,5.0,NaN,86.0,52.0,NaN,NaN,NaN
A verified traveler,426.0,NaN,342.0,NaN,NaN,NaN,404.0,NaN
Anonymous,NaN,NaN,NaN,NaN,NaN,97.0,NaN,67.0
Christine,NaN,NaN,NaN,NaN,NaN,4.0,4.0,3.0
David,NaN,NaN,NaN,NaN,4.0,19.0,NaN,NaN
James,NaN,NaN,3.0,NaN,NaN,4.0,NaN,4.0
Jennifer,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
John,5.0,NaN,5.0,NaN,NaN,3.0,5.0,NaN
Lisa,10.0,NaN,NaN,5.0,NaN,NaN,NaN,2.0


In [19]:
user_enc = LabelEncoder()
data['User'] = user_enc.fit_transform(data['Users'].values)
n_users = data['User'].nunique()

In [21]:
item_enc = LabelEncoder()
data['Hotels'] = item_enc.fit_transform(data['Hotel'].values)
n_hotels = data['Hotels'].nunique()

In [22]:
data['Rating'] = data['Rating'].values.astype(np.float32)
min_rating = min(data['Rating'])
max_rating = max(data['Rating'])

In [23]:
n_users, n_hotels, min_rating, max_rating


(17852, 2912, 1.0, 5.0)

In [24]:
X = data[['User', 'Hotels']].values
y = data['Rating'].values

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [26]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((21834, 2), (2427, 2), (21834,), (2427,))

In [27]:
n_factors = 50
X_train_array = [X_train[:, 0], X_train[:, 1]]
X_test_array = [X_test[:, 0], X_test[:, 1]]

In [28]:
from keras.models import Model
from keras.layers import Input, Reshape, Dot
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.regularizers import l2

In [31]:
def RecommenderV1(n_users, n_hotels, n_factors):
    user = Input(shape=(1,))
    u = Embedding(n_users, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(user)
    u = Reshape((n_factors,))(u)
    
    hotel = Input(shape=(1,))
    m = Embedding(n_movies, n_factors, embeddings_initializer='he_normal',
                  embeddings_regularizer=l2(1e-6))(hotel)
    m = Reshape((n_factors,))(m)
    
    x = Dot(axes=1)([u, m])
    model = Model(inputs=[user, hotel], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [32]:
model = RecommenderV1(n_users, n_hotels, n_factors)
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 1, 50)        892600      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 1, 50)        145600      input_4[0][0]                    
_______________________________________________________________________________________

In [33]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


342/342 [==============================] - 3s 9ms/step - loss: 17.2527 - val_loss: 16.7516
Epoch 2/5
342/342 [==============================] - 3s 9ms/step - loss: 15.9439 - val_loss: 16.0534
Epoch 3/5
342/342 [==============================] - 3s 9ms/step - loss: 13.9922 - val_loss: 15.6122
Epoch 4/5
342/342 [==============================] - 3s 9ms/step - loss: 11.0429 - val_loss: 15.2228
Epoch 5/5
342/342 [==============================] - 3s 9ms/step - loss: 7.9678 - val_loss: 14.9382


In [34]:
from keras.layers import Add, Activation, Lambda
class EmbeddingLayer:
    def __init__(self, n_items, n_factors):
        self.n_items = n_items
        self.n_factors = n_factors
    
    def __call__(self, x):
        x = Embedding(self.n_items, self.n_factors, embeddings_initializer='he_normal',
                      embeddings_regularizer=l2(1e-6))(x)
        x = Reshape((self.n_factors,))(x)
        return x

In [36]:
def RecommenderV2(n_users, n_hotels, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    ub = EmbeddingLayer(n_users, 1)(user)
    
    hotel = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(hotel)
    mb = EmbeddingLayer(n_movies, 1)(hotel)
    x = Dot(axes=1)([u, m])
    x = Add()([x, ub, mb])
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, hotel], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [37]:
model = RecommenderV2(n_users, n_hotels, n_factors, min_rating, max_rating)
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_4 (Embedding)         (None, 1, 50)        892600      input_5[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 1, 50)        145600      input_6[0][0]                    
_______________________________________________________________________________________

In [38]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


342/342 [==============================] - 3s 9ms/step - loss: 2.3232 - val_loss: 2.1733
Epoch 2/5
342/342 [==============================] - 3s 8ms/step - loss: 2.0148 - val_loss: 2.0263
Epoch 3/5
342/342 [==============================] - 3s 9ms/step - loss: 1.5203 - val_loss: 1.9291
Epoch 4/5
342/342 [==============================] - 3s 8ms/step - loss: 1.0235 - val_loss: 1.8823
Epoch 5/5
342/342 [==============================] - 3s 8ms/step - loss: 0.6984 - val_loss: 1.8546


In [39]:
from keras.layers import Concatenate, Dense, Dropout
def RecommenderNet(n_users, n_hotels, n_factors, min_rating, max_rating):
    user = Input(shape=(1,))
    u = EmbeddingLayer(n_users, n_factors)(user)
    
    hotel = Input(shape=(1,))
    m = EmbeddingLayer(n_movies, n_factors)(hotel)
    
    x = Concatenate()([u, m])
    x = Dropout(0.05)(x)
    
    x = Dense(10, kernel_initializer='he_normal')(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    
    x = Dense(1, kernel_initializer='he_normal')(x)
    x = Activation('sigmoid')(x)
    x = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(x)
    model = Model(inputs=[user, hotel], outputs=x)
    opt = Adam(lr=0.001)
    model.compile(loss='mean_squared_error', optimizer=opt)
    return model

In [40]:
model = RecommenderNet(n_users, n_movies, n_factors, min_rating, max_rating)
model.summary()

Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_8 (InputLayer)            [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 1, 50)        892600      input_7[0][0]                    
__________________________________________________________________________________________________
embedding_9 (Embedding)         (None, 1, 50)        145600      input_8[0][0]                    
_______________________________________________________________________________________

In [41]:
history = model.fit(x=X_train_array, y=y_train, batch_size=64, epochs=5,
                    verbose=1, validation_data=(X_test_array, y_test))

Epoch 1/5


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


342/342 [==============================] - 3s 10ms/step - loss: 1.5386 - val_loss: 1.1148
Epoch 2/5
342/342 [==============================] - 3s 9ms/step - loss: 0.9795 - val_loss: 1.2101
Epoch 3/5
342/342 [==============================] - 3s 9ms/step - loss: 0.6429 - val_loss: 1.1915
Epoch 4/5
342/342 [==============================] - 3s 9ms/step - loss: 0.5275 - val_loss: 1.2139
Epoch 5/5
342/342 [==============================] - 3s 9ms/step - loss: 0.4485 - val_loss: 1.2301
